Notebook to test and experiment with IG Classes
There are some trials to check HikerAPI, but there are no test of the post and comment treatment using Instagrapy

### Variable setting and other initial imports

In [1]:
"""
main.py:
Example usage of the Instagram client, post manager, and music features to automate content posting.
"""

import os, pprint
from ig_auth import authenticate_and_save_session
from ig_client import IgClient
from ig_post_manager import IgPostManager


### 1. Authentication (If you don't have a session file yet)

In [2]:
# 1. Authentication (Always attempt authentication first)
session_file = "cl_ig.pkl"
if not os.path.exists(session_file):  # Check if session file exists
    authenticate_and_save_session(session_file)

# Now, you should have a valid session, so create the client
insta_client = IgClient(session_file)
post_manager = IgPostManager(insta_client)


In [3]:
pprint.pprint(insta_client)

#### Initialize a list to store IgPost objects

In [4]:
ig_posts = []

### Test the basic APIs for p-mgmt

In [5]:
# 2. Post a Photo:
photo_path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG-20160212-WA0018.jpeg"
photo_caption = """Chasing sunsets along the enchanting Malecon in Miraflores, Lima. 🌅 
Embracing the golden hues as the sun dips below the horizon, 
painting the sky in a masterpiece of colors. 
Where's your favorite sunset spot? Share below! 

#SunsetMagic #MirafloresSunset 
#LimaSunsets #MaleconViews #GoldenHourGram #PeruvianSunsets #TravelDiaries #SunsetChasers #NaturePhotography #TravelGram #WanderlustAdventures"""

location_pk = 61047499  # Malecón Cisneros - Miraflores
ig_post = post_manager.upload_photo(photo_path, photo_caption, location_pk=location_pk)
ig_posts.append(ig_post) 


In [11]:
from IPython.display import display

display(ig_posts)               # Display formatted contents
  

In [ ]:

# 3. Post a Video:
video_path = "path/to/your/video.mp4"  # Replace with your actual video path
video_caption = "Here's a quick recap of my weekend adventure! #travel #fun"
post_manager.upload_video(video_path, video_caption)


In [ ]:

# 4. Post a Carousel (Album) of Photos and Videos:
paths = ["path/to/photo1.jpg", "path/to/photo2.jpg", "path/to/video.mp4"]
album_caption = "Memories from my recent trip! #vacation #beach"
post_manager.upload_album(paths, album_caption)


In [ ]:

# 5. Upload a Reel with Music:
reel_video_path = "path/to/your/reel.mp4"
reel_caption = "Having fun with this new trend! #dance #music"


In [ ]:

# Option A: Search for music on Instagram
search_query = "Chill vibes"
music_results = post_manager.search_music(search_query)
selected_track_id = music_results[0].id  # Choose the first result (be sure to validate!)


In [ ]:

# Option B: Use your own music
# music_path = "path/to/your/music.mp3"
# selected_track_id = post_manager.client.music_upload(music_path)

post_manager.upload_reel_with_music(reel_video_path, selected_track_id, reel_caption)


### Test the more advance p-mgmt APIs

In [ ]:
from ig_client import IgClient
from ig_post_manager import IgPostManager

insta_client = IgClient()
post_manager = IgPostManager(insta_client)

# Upload video with a music track
video_path = "path/to/your/video.mp4"
music_path = "path/to/your/music.mp3"
post = post_manager.upload_video_with_music(video_path, music_path, "My awesome video with music!")


# Search for music
music_results = post_manager.search_music("Upbeat pop")
for track in music_results:
    print(f"{track.id}: {track.title} - {track.artist}")

# Select a track by its ID and upload a reel
selected_track_id = music_results[0].id  # Choose the first track
post = post_manager.upload_reel_with_music("path/to/your/reel.mp4", selected_track_id, "My fun reel!")


### Several Test for Hiker API to get info

# NEED TO CONVERT INTO METHODS OF A CLASS

In [ ]:
%pip install --upgrade hikerapi




In [ ]:
from hikerapi import Client
hiker_api_key = os.environ["HikerAPI_key"]
cl = Client(token=hiker_api_key)  

In [ ]:
print(type(cl))

In [ ]:
print(hiker_api_key)

### Hiker API - Get user info

In [ ]:


user_info = cl.user_by_username_v2("huelvafoodie")

pprint(user_info)

## Extract the most important Huelva influencers profiles by name
### store them in a JSON file

In [ ]:
import os
import json
from hikerapi import Client

def fetch_and_save_profiles(usernames, output_file="instagram_profiles.json"):
    """Fetches user profiles from HikerAPI and saves the data to a JSON file.

    Args:
        usernames: A list of Instagram usernames to fetch.
        output_file: The path to the JSON file where data will be saved.
    """
    
    # Load your HikerAPI key securely from an environment variable
    hiker_api_key = os.environ.get("HikerAPI_key")
    if not hiker_api_key:
        raise ValueError("HikerAPI_key environment variable not found.")

    # Create the HikerAPI client
    cl = Client(token=hiker_api_key)

    all_profile_data = []

    for username in usernames:
        try:
            user_info = cl.user_by_username_v2(username)

            # If the request is successful, add it to the list
            if user_info['status'] == 'ok':
                all_profile_data.append(user_info)
            else:
                print(f"Error fetching data for {username}: {user_info.get('error')}")
        except Exception as e:
            print(f"Error fetching data for {username}: {e}")

    # Save the data to a JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_profile_data, f, ensure_ascii=False, indent=4)




In [ ]:
# call the function to fetch the information
usernames_to_fetch = ["capturolavida_emn",
                      "huelvafoodie",
                      "laesenciadehuelva",
                      "huelvagram",
                      "agendahuelva",
                      "huelva_secreta",
                      "huelva.explore",
                      "huelva24com",
                      "huelvainformacion",
                      "huelvahoy",
                      "huelvaoriginal"]

# Call the function to fetch and save profiles
fetch_and_save_profiles(usernames_to_fetch)    # add arg for output_file - default "instagram_profiles.json"

## Extract relevant information from the JSON and store it in a pandas data frame
## and, latter, store in a .csv file
## to access it with Excel or Google Sheets

In [ ]:
import os
import json
import pandas as pd

def extract_profile_data_from_json(input_file="instagram_profiles.json", output_file="instagram_profiles.csv"):
    """Loads Instagram profile data from a JSON file, extracts relevant fields,
    creates a Pandas DataFrame, and exports it to a CSV file.

    Args:
        input_file: The path to the JSON file containing the profile data.
        output_file: The path to save the CSV file.
    """

    with open(input_file, "r", encoding="utf-8") as f:
        all_profile_data = json.load(f)

    extracted_data = []

    for profile in all_profile_data:
        if profile["status"] == "ok":
            user = profile["user"]
            extracted_data.append({
                "username": user["username"],
                "full_name": user["full_name"],
                "instagram_id": user.get("pk", user.get("pk_id")),  # Extract the ID (either "pk" or "pk_id")
                "category": user["category"],
                "biography": user["biography"],
                "followers": user["follower_count"],
                "following": user["following_count"],
                "is_private": user["is_private"],
                "is_business": user["is_business"],
                "is_verified": user["is_verified"],
                "media_count": user["media_count"],
                "fb_page_id": str(user["page_id"]),
                "fb_page_name": user["page_name"],
                "profile_pic_url": user["profile_pic_url"],
                "external_url": user.get("external_url"), # Use get() to handle missing keys
                "bio_links": [link["url"] for link in user.get("bio_links", [])] # Extract URLs from bio links
            })

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(extracted_data)

    # Export the DataFrame to a CSV file (for Google Sheets)
    df.to_csv(output_file, index=False)

# Example Usage: 
extract_profile_data_from_json()    # Using default file names
                                    # input_file="instagram_profiles.json",
                                    # output_file="instagram_profiles.csv"


